# _**Case 1 — The Missing Order**_

**Story:**

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">A long-time client insists they placed an order that never arrived. You dig through the company records searching for any orders that were created but never shipped. Somewhere, a missing ShipToDate</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">&nbsp;might reveal the truth.</span>

In [4]:
SELECT 
    c.CustomerCompanyName,     -- customer’s company
    c.CustomerContactName,     -- their contact person
    o.OrderId,                 -- order ID
    o.OrderDate,               -- when order was placed
    o.ShipToDate               -- when (or if) it shipped
FROM Sales.[Order] AS o
JOIN Sales.Customer AS c 
    ON o.CustomerId = c.CustomerId
WHERE o.ShipToDate IS NULL;    -- find orders that never shipped


(21 rows affected)

Total execution time: 00:00:00.036

CustomerCompanyName,CustomerContactName,OrderId,OrderDate,ShipToDate
Customer THHDP,"Kane, John",11008,2022-04-08,NULL
Customer LWGMD,"Gaffney, Lawrie",11019,2022-04-13,NULL
Customer PSQUZ,"Lupu, Cornel",11039,2022-04-21,NULL
Customer YSIQX,"Krishnan, Venky",11040,2022-04-22,NULL
Customer EEALV,"Culp, Scott",11045,2022-04-23,NULL
Customer XIIWM,"Litton, Tim",11051,2022-04-27,NULL
Customer PSNMQ,"Ray, Mike",11054,2022-04-28,NULL
Customer XHXJV,"Poland, Carole",11058,2022-04-29,NULL
Customer QVEPD,"Garden, Euan",11059,2022-04-29,NULL
Customer YSIQX,"Krishnan, Venky",11061,2022-04-30,NULL


# _**Case 2 — The Overpriced Products**_

**Story:**  

The finance team whispers of a suspicious invoice—someone may have overcharged for a product. You must hunt down the orders where prices seem unusually high and expose the most expensive sale.

In [5]:
SELECT TOP 10
    p.ProductName,                 -- name of the product sold
    od.UnitPrice,                  -- price per unit for that product in the order
    c.CustomerCompanyName,         -- company that purchased the product
    o.OrderDate                    -- date the order was placed
FROM Sales.OrderDetail AS od
JOIN Production.Product AS p
    ON od.ProductId = p.ProductId  -- link each order detail to its product info
JOIN Sales.[Order] AS o
    ON od.OrderId = o.OrderId      -- link order details to the order header
JOIN Sales.Customer AS c
    ON o.CustomerId = c.CustomerId -- link order to customer info
ORDER BY od.UnitPrice DESC;        -- sort so highest-priced sales appear first

(10 rows affected)

Total execution time: 00:00:00.034

ProductName,UnitPrice,CustomerCompanyName,OrderDate
Product QDOMO,263.50,Customer VONTK,2021-04-25
Product QDOMO,263.50,Customer IRRVL,2021-05-19
Product QDOMO,263.50,Customer IBVRG,2021-05-19
Product QDOMO,263.50,Customer YSIQX,2021-07-31
Product QDOMO,263.50,Customer HGVLZ,2021-09-17
Product QDOMO,263.50,Customer IBVRG,2021-12-18
Product QDOMO,263.50,Customer LCYBZ,2021-12-30
Product QDOMO,263.50,Customer YSIQX,2022-01-06
Product QDOMO,263.50,Customer GLLAG,2022-01-06
Product QDOMO,263.50,Customer LWGMD,2022-01-13


# _**Case 3 —**_ **_The Supplier Who Shipped Nothing_**

**Story:**  
A proud supplier swears they’ve been shipping stock nonstop, yet the warehouse logs are silent. You set out to uncover which suppliers have never once had their products included in an order.

In [11]:

SELECT s.SupplierId                          -- identify the supplier
FROM Production.Supplier AS s
WHERE NOT EXISTS (
    SELECT 1
    FROM Production.Product AS p
    JOIN Sales.OrderDetail AS od
         ON od.ProductId = p.ProductId
    WHERE p.SupplierId = s.SupplierId        -- any order line for this supplier?
);

(0 rows affected)

Total execution time: 00:00:00.023

SupplierId


# _**Case 4 — Customers from London**_

**Story:**  

Marketing plans a new campaign targeting urban clients. Your task: identify every customer based in London before the next mailer goes out.

In [7]:
SELECT 
    CustomerCompanyName,           -- name of the company
    CustomerContactName,           -- contact person at the company
    CustomerCity,                  -- city name (should be 'London')
    CustomerCountry                -- customer’s country
FROM Sales.Customer
WHERE CustomerCity = 'London';     -- filter to show only London-based customers

(6 rows affected)

Total execution time: 00:00:00.038

CustomerCompanyName,CustomerContactName,CustomerCity,CustomerCountry
Customer GYBBY,"Birkby, Dana",London,UK
Customer RFNQC,"Boseman, Randall",London,UK
Customer HFBZG,"Cunningham, Conor",London,UK
Customer UBHAU,"Jaffe, David",London,UK
Customer GCJSG,"Mallit, Ken",London,UK
Customer AHPOP,"Welcker, Brian",London,UK


# **_Case 5 —_** **_The Employee of Suspense_**

**Story:**  
Human Resources suspects one salesperson has been quietly coasting. You need to find the employees who’ve processed almost no orders, before the quarterly reviews begin.

In [10]:
SELECT 
    e.EmployeeFirstName + ' ' + e.EmployeeLastName AS EmployeeName,
    COUNT(o.OrderId) AS OrdersHandled
FROM HumanResources.Employee AS e
LEFT JOIN Sales.[Order] AS o
    ON e.EmployeeId = o.EmployeeId            -- link orders to employees
GROUP BY e.EmployeeFirstName, e.EmployeeLastName
HAVING COUNT(o.OrderId) < 3                   -- fewer than 3 orders
ORDER BY OrdersHandled;


(0 rows affected)

Total execution time: 00:00:00.058

EmployeeName,OrdersHandled


# **_Case 6 — The Mislabeled Product_**

**Story:**  

A crate marked “Condiments” arrived filled with tea bags and cocoa tins. Someone mixed up the categories, and it’s up to you to find products whose names sound like beverages but aren’t filed under _Beverages_.

In [12]:
SELECT 
    p.ProductName,                 -- product name
    cat.CategoryName               -- category where the product is currently listed
FROM Production.Product AS p
JOIN Production.Category AS cat
  ON p.CategoryId = cat.CategoryId -- link each product to its category
WHERE (p.ProductName LIKE '%Tea%'  -- search for product names containing 'Tea'
    OR p.ProductName LIKE '%Coffee%'   -- ...or 'Coffee'
    OR p.ProductName LIKE '%Juice%')   -- ...or 'Juice'
  AND cat.CategoryName NOT LIKE '%Beverage%'; -- exclude correctly labeled Beverage items

(0 rows affected)

Total execution time: 00:00:00.012

ProductName,CategoryName


# **_Case 7 — The Global Sales Race_**

**Story:**  
The CEO demands to know where the company’s real profits come from. You’ll have to tally every shipment’s destination to discover which countries bring in the most revenue.

In [13]:
SELECT 
    o.ShipToCountry,                               -- destination country
    SUM(od.UnitPrice * od.Quantity) AS TotalRevenue,  -- total value of items shipped (no discount column exists)
    COUNT(DISTINCT o.OrderId) AS OrdersCount          -- number of unique orders shipped to that country
FROM Sales.OrderDetail AS od
JOIN Sales.[Order] AS o
  ON od.OrderId = o.OrderId                        -- connect order details to their parent orders
GROUP BY o.ShipToCountry                           -- group by destination country
ORDER BY TotalRevenue DESC;                        -- show top-grossing countries first


(21 rows affected)

Total execution time: 00:00:00.032

ShipToCountry,TotalRevenue,OrdersCount
USA,263566.98,122
Germany,244640.63,122
Austria,139496.63,40
Brazil,114968.48,83
France,85498.76,77
Venezuela,60814.89,46
UK,60616.51,56
Sweden,59523.70,37
Ireland,57317.39,19
Canada,55334.10,30
